In [214]:
import numpy as np
import pandas as  pd
import requests
import ast



In [215]:
a=pd.read_csv(r'D:\movie project\tmdb_5000_credits.csv')
b=pd.read_csv(r'D:\movie project\tmdb_5000_movies.csv')

In [216]:
a=a.merge(b,on='title')

In [217]:
a=a.drop(['homepage','tagline'],axis=1)

In [218]:
a.duplicated().sum()

0

In [219]:
genre=a['genres']

In [220]:
def convert(obj):
    names=[]
    for i in ast.literal_eval(obj):
        names.append(i['name'])
    return names


In [221]:
a['genres']=a['genres'].apply(convert)

In [222]:
a['keywords']=a['keywords'].apply(convert)

In [223]:
def convert_cast(obj):
    l=[]
    for i in ast.literal_eval(obj)[0:3]:
      l.append(i['name'])
    return l


In [224]:
a['cast']=a['cast'].apply(convert_cast)

In [225]:
a.shape

(4809, 21)

In [226]:
def director(obj):
 l=[]
 for i in ast.literal_eval(obj):
    if i['job']=='Director':
        l.append(i['name'])
 return l
        


In [227]:
a['crew']=a['crew'].apply(director)

In [228]:
a['overview']=a['overview'].apply(lambda x:str(x).split())


In [229]:
def space(obj):
 word=obj
 for i in range(len(word)):
  word[i]=word[i].replace(' ','')
 return word




In [230]:
a['cast']=a['cast'].apply(space)

In [231]:
a['crew']=a['crew'].apply(space)
a['keywords']=a['keywords'].apply(space)
a['genres']=a['genres'].apply(space)

In [232]:
a['tags']=a['overview']+a['genres']+a['keywords']+a['cast']+a['crew']

In [233]:
new_df=a[['movie_id','title','tags']]

In [234]:
new_df['tags']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: tags, Length: 4809, dtype: object

In [236]:
a['tags']=a['tags'].apply(lambda x:' '.join(x))

In [237]:
new_df['tags']=new_df['tags'].apply(lambda x:str(x).lower())

C:\Users\rahul\AppData\Local\Temp\ipykernel_12508\983240207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:str(x).lower())


In [238]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


In [239]:
import nltk
from nltk.stem import PorterStemmer
ps=PorterStemmer()


In [240]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [241]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\rahul\AppData\Local\Temp\ipykernel_12508\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [242]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [243]:
from sklearn.metrics.pairwise import cosine_similarity

In [244]:
distances=cosine_similarity(vectors)

In [ ]:
def recommand(movie):
    id=new_df[new_df['title']==movie].index[0]
    distance=distances[id]
    movie_list=sorted(enumerate(distance),reverse=True,key=lambda x:x[1])[:5]
    indx=[]
    for i in movie_list:
        indx.append(i[0])
    return indx


In [245]:
for i in list(recommand('Batman Begins')):
    print(new_df.iloc[i].title)


Batman Begins
The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin


In [247]:
import pickle
pickle.dump(new_df,open('moviedata.pkl','wb'))
pickle.dump(distances,open('distsnces.pkl','wb'))